## Pokemon Webscraper
### Webscraping Gen 1 Pokemon


## Import Modules

In [2]:
import numpy as np
import pandas as pd


In [3]:
import requests

### Get Webpage

In [85]:
URL = 'https://pokemondb.net/pokedex/stats/gen1'
page = requests.get(URL)
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/91.0.4472.124'}
type(page)

requests.models.Response

In [72]:
print(page.text)

<!DOCTYPE html>
<html lang="en">
<head>
	<meta charset="utf-8">
	<title>Generation 1 new Pokémon stats | Pokémon Database</title>

	<link rel="preconnect" href="https://img.pokemondb.net">
	<link rel="preconnect" href="https://s.pokemondb.net">
	<link rel="preload" href="/static/fonts/fira-sans-v17-latin-400.woff2" as="font" type="font/woff2" crossorigin>
	<link rel="preload" href="/static/fonts/fira-sans-v17-latin-400i.woff2" as="font" type="font/woff2" crossorigin>
	<link rel="preload" href="/static/fonts/fira-sans-v17-latin-600.woff2" as="font" type="font/woff2" crossorigin>
		<link rel="stylesheet" href="/static/css/pokemondb-96b23815d7.css">

	<meta name="viewport" content="width=device-width, initial-scale=1">

	<meta property="og:description" name="description" content="List of new Pokémon introduced in Gen 1 (Red/Blue/Yellow) along with their stats.">
	<link rel="canonical" href="https://pokemondb.net/pokedex/stats/gen1">
	<meta property="og:url" content="https://pokemondb.net/

### Parse HTML with Beautiful Soup

In [73]:
from bs4 import BeautifulSoup

In [74]:
soup  = BeautifulSoup(page.content, 'html.parser')

In [75]:
dir(soup)

['ASCII_SPACES',
 'DEFAULT_BUILDER_FEATURES',
 'DEFAULT_INTERESTING_STRING_TYPES',
 'EMPTY_ELEMENT_EVENT',
 'END_ELEMENT_EVENT',
 'NO_PARSER_SPECIFIED_WARNING',
 'ROOT_TAG_NAME',
 'START_ELEMENT_EVENT',
 'STRING_ELEMENT_EVENT',
 '__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_clone',
 '_decode_markup',
 '_event_stream',
 '_feed',
 '_find_all',
 '_find_one',
 '_format_tag',
 '_indent_string',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descenda

In [77]:
results = soup.find(id = 'main')
print(results.prettify())
gen = results.find('table')

<main class="main-content grid-container" id="main">
 <h1>
  Generation 1 Pokémon stats
 </h1>
 <div class="panel panel-intro">
  <p>
   This is a list of all the Pokémon from Generation 1 (Red, Blue &amp; Yellow). It includes the base stats of every new Pokémon and form introduced.
  </p>
 </div>
 <ul class="list-nav panel panel-nav">
  <li class="list-nav-title">
   Other generations
  </li>
  <li>
   <b>
    1
   </b>
  </li>
  <li>
   <a href="/pokedex/stats/gen2" title="Generation 2 Pokémon stats">
    2
   </a>
  </li>
  <li>
   <a href="/pokedex/stats/gen3" title="Generation 3 Pokémon stats">
    3
   </a>
  </li>
  <li>
   <a href="/pokedex/stats/gen4" title="Generation 4 Pokémon stats">
    4
   </a>
  </li>
  <li>
   <a href="/pokedex/stats/gen5" title="Generation 5 Pokémon stats">
    5
   </a>
  </li>
  <li>
   <a href="/pokedex/stats/gen6" title="Generation 6 Pokémon stats">
    6
   </a>
  </li>
  <li>
   <a href="/pokedex/stats/gen7" title="Generation 7 Pokémon stats">
 

### Now that we have the page, let's scrape results

First, I will need a way to find the gen of all kinds of pokemon. It is hidden in the results page in an href, so we will scrape it from there

In [55]:
def get_gen(results, gen):
    gen = results.find('a', href = f'/pokedex/stats/{gen}')
    gen.text
    gen_trim = gen.text.strip()
    gen = gen_trim[0:5]
    return gen

In [80]:
poke_table = soup.find('table',id = 'pokedex')
poke_table.prettify()


'<table class="data-table" id="pokedex">\n <thead>\n  <tr>\n   <th class="sorting" data-sort-type="int">\n    <div class="sortwrap">\n     #\n    </div>\n   </th>\n   <th class="sorting" data-sort-type="string">\n    <div class="sortwrap">\n     Name\n    </div>\n   </th>\n   <th>\n    <div class="sortwrap">\n     Type\n    </div>\n   </th>\n   <th class="sorting" data-sort-default="desc" data-sort-type="int">\n    <div class="sortwrap">\n     Total\n    </div>\n   </th>\n   <th class="sorting" data-sort-default="desc" data-sort-type="int">\n    <div class="sortwrap">\n     HP\n    </div>\n   </th>\n   <th class="sorting" data-sort-default="desc" data-sort-type="int">\n    <div class="sortwrap">\n     Attack\n    </div>\n   </th>\n   <th class="sorting" data-sort-default="desc" data-sort-type="int">\n    <div class="sortwrap">\n     Defense\n    </div>\n   </th>\n   <th class="sorting" data-sort-default="desc" data-sort-type="int">\n    <div class="sortwrap">\n     Sp. Atk\n    </div>\

In [82]:
types_df = pd.DataFrame([
    {'name': t} for t in ['Normal', 'Fire', 'Water', 'Electric', 'Grass', 'Ice', 
                          'Fighting', 'Poison', 'Ground', 'Flying', 'Psychic', 
                          'Bug', 'Rock', 'Ghost', 'Dragon']
])
types_df

,name
0,Normal
1,Fire
2,Water
3,Electric
4,Grass
5,Ice
6,Fighting
7,Poison
8,Ground
9,Flying


In [83]:

pokemon_data = []

for row in poke_table.find_all('tr')[1:]:
    cells = row.find_all('td')
    types = [a.text.strip() for a in cells[2].find_all('a')]
    pokemon_data.append({
        'id':cells[0].text.strip(),
        'name': cells[1].find('a').text.strip(),
        'type1' : types[0] if type else None,
        'type2' : types[1] if len(types) > 1 else None, 
        'hp' : int(cells[4].text),
        'attack': int(cells[5].text),
        'defense': int(cells[6].text),
        'sp_attack': int(cells[7].text),
        'sp_defense': int(cells[8].text),
        'speed': int(cells[9].text),
        'sprite_url': f"https://img.pokemondb.net/sprites/red-blue/normal/{cells[1].find('a').text.lower()}.png",
        'detail_url': 'https://pokemondb.net' + cells[1].find('a')['href']
    })
pokemon_df = pd.DataFrame(pokemon_data)
        


In [84]:
pokemon_df = pd.DataFrame(pokemon_data)
pokemon_df

,id,name,type1,type2,hp,attack,defense,sp_attack,sp_defense,speed,sprite_url,detail_url
0,0001,Bulbasaur,Grass,Poison,45,49,49,65,65,45,https://img.pokemondb.net/sprites/red-blue/nor...,https://pokemondb.net/pokedex/bulbasaur
1,0002,Ivysaur,Grass,Poison,60,62,63,80,80,60,https://img.pokemondb.net/sprites/red-blue/nor...,https://pokemondb.net/pokedex/ivysaur
2,0003,Venusaur,Grass,Poison,80,82,83,100,100,80,https://img.pokemondb.net/sprites/red-blue/nor...,https://pokemondb.net/pokedex/venusaur
3,0004,Charmander,Fire,None,39,52,43,60,50,65,https://img.pokemondb.net/sprites/red-blue/nor...,https://pokemondb.net/pokedex/charmander
4,0005,Charmeleon,Fire,None,58,64,58,80,65,80,https://img.pokemondb.net/sprites/red-blue/nor...,https://pokemondb.net/pokedex/charmeleon
...,...,...,...,...,...,...,...,...,...,...,...,...
146,0147,Dratini,Dragon,None,41,64,45,50,50,50,https://img.pokemondb.net/sprites/red-blue/nor...,https://pokemondb.net/pokedex/dratini
147,0148,Dragonair,Dragon,None,61,84,65,70,70,70,https://img.pokemondb.net/sprites/red-blue/nor...,https://pokemondb.net/pokedex/dragonair
148,0149,Dragonite,Dragon,Flying,91,134,95,100,100,80,https://img.pokemondb.net/sprites/red-blue/nor...,https://pokemondb.net/pokedex/dragonite
149,0150,Mewtwo,Psychic,None,106,110,90,154,90,130,https://img.pokemondb.net/sprites/red-blue/nor...,https://pokemondb.net/pokedex/mewtwo


### Scrape Moves

Now that we have a data set containing all generation 1 pokemon and their stats, we can scrape all moves for  each pokemon

In [87]:
moves_df = pd.DataFrame()
url = "https://pokemondb.net/move/generation/1"
response = requests.get(url, headers = headers, timeout = 10)
response.raise_for_status()
soup = BeautifulSoup(response.text, 'html.parser')
moves_table = soup.find('table', id = 'moves')
moves_df = pd.read_html(str(moves_table))[0]
moves_df = moves_df[['Name','Type','Cat.', 'Power', 'Acc.','PP']]
moves_df.columns = ['name','type','category','power','accuracy','pp']

moves_df


C:\Users\Mason\AppData\Local\Temp\ipykernel_3920\3581632406.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  moves_df = pd.read_html(str(moves_table))[0]


,name,type,category,power,accuracy,pp
0,Absorb,Grass,NaN,20,100,25
1,Acid,Poison,NaN,40,100,30
2,Acid Armor,Poison,NaN,—,—,20
3,Agility,Psychic,NaN,—,—,30
4,Amnesia,Psychic,NaN,—,—,20
...,...,...,...,...,...,...
160,Waterfall,Water,NaN,80,100,15
161,Whirlwind,Normal,NaN,—,—,20
162,Wing Attack,Flying,NaN,60,100,35
163,Withdraw,Water,NaN,—,—,40


In [91]:
pokemon_df.to_csv('data/processed/gen1_pokemon.csv',index = False)
types_df.to_csv('data/processed/gen1_types.csv', index = False)
moves_df.to_csv('data/processed/gen1_moves.csv', index = False)

### Scraping Pokemon-specific Data

We have our base informatino on pokemon, types and moves fro our database, now we need to fill in moves for each pokemon as well as abilities

In [101]:
pokemon_df['detail_url']

0       https://pokemondb.net/pokedex/bulbasaur
1         https://pokemondb.net/pokedex/ivysaur
2        https://pokemondb.net/pokedex/venusaur
3      https://pokemondb.net/pokedex/charmander
4      https://pokemondb.net/pokedex/charmeleon
                         ...                   
146       https://pokemondb.net/pokedex/dratini
147     https://pokemondb.net/pokedex/dragonair
148     https://pokemondb.net/pokedex/dragonite
149        https://pokemondb.net/pokedex/mewtwo
150           https://pokemondb.net/pokedex/mew
Name: detail_url, Length: 151, dtype: object

In [120]:
pokemon_moves_data = []
gen_str = '/moves/1'
for index, row in pokemon_df.iterrows():
    try:
        url = row['detail_url'] + gen_str
        response = requests.get(url, headers = headers, timeout = 10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        move_section = soup.find('h3', string = 'Moves learnt by level up')
        if move_section:
            move_table = move_section.find_next('tbody')
            if move_table:
                for tr in move_table.find_all('tr'):
                    cell = tr.find_all('td')
                    move_name = cell[1].text.strip()
                    move_type = cell[2].text.strip()
                    move_level = cell[0].text.strip()
                    pokemon_moves_data.append({
                        'pokemon_id': row['id'],
                        'move_name' : move_name,
                        'move_type' : move_type,
                        'learn_method' : f'Level {move_level}'
                    })
            else:
                print('did not work')
        hm_section = soup.find('h3', string = 'Moves learnt by HM')
        if hm_section:
            hm_table = hm_section.find_next('tbody')
            if hm_table:
                for tr in hm_table.find_all('tr'):
                    cell = tr.find_all('td')
                    move_name = cell[1].text.strip()
                    move_type = cell[2].text.strip()
                  
                    pokemon_moves_data.append({
                        'pokemon_id': row['id'],
                        'move_name' : move_name,
                        'move_type' : move_type,
                        'learn_method' : 'HM'
                    })
        tm_section = soup.find('h3', string = 'Moves learnt by TM')
        if tm_section:
            tm_table = tm_section.find_next('tbody')
            if tm_table:
                for tr in tm_table.find_all('tr'):
                    cell = tr.find_all('td')
                    move_name = cell[1].text.strip()
                    move_type = cell[2].text.strip()
                    pokemon_moves_data.append({
                        'pokemon_id': row['id'],
                        'move_name' : move_name,
                        'move_type' : move_type,
                        'learn_method' : 'TM'
                    })
            

    except Exception as e:
        print(f"Error scraping {row['name']}: {e}")

In [121]:
pokemon_moves_df = pd.DataFrame(pokemon_moves_data)
pokemon_moves_df

,pokemon_id,move_name,move_type,learn_method
0,0001,Growl,Normal,Level 1
1,0001,Tackle,Normal,Level 1
2,0001,Leech Seed,Grass,Level 7
3,0001,Vine Whip,Grass,Level 13
4,0001,PoisonPowder,Poison,Level 20
...,...,...,...,...
4642,0151,Psywave,Psychic,TM
4643,0151,Explosion,Normal,TM
4644,0151,Rock Slide,Rock,TM
4645,0151,Tri Attack,Normal,TM


In [122]:
pokemon_moves_df.to_csv('data/processed/gen1_pokemon_moves.csv', index = False)